In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import certifi
import urllib3
import html5lib
import math
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np

Import HTML File

In [2]:
URL = "https://www.ttilgb.com/uiAgs03Action/openScreen.do"
res = requests.get(URL, 'lxml', verify=False)
soup = BeautifulSoup(res.content, 'lxml')

Find needed Table from web page
*To find needed table look for children or you Soup and prettify

In [3]:
#Or children = table.findChildren()
#for child in children:
    #print (child)

In [4]:
#Find table
table1 = soup.find_all('tr')[0] 

In [5]:
df1 = pd.read_html(str(table1))
#print (df1)

In [6]:
#filter what we need
table2 = soup.find_all("td", { "class" : "h23" })

In [7]:
df2 = pd.read_html(str(table2))
#df2

In [8]:
#soupparser = BeautifulSoup(res.content, 'html.parser')
#print(soupparser.prettify()) 

In [9]:
#filter what we need
table = soup.find('tbody')

Extract and clean data

In [10]:
#pull data cleanly as possible, tried several loop functions to pull, 
#but with the muliple rowspans and colspans was taking to long and decided to clean with pandas
res = []
row = []

for tr in table.find_all('tr'):
     for td in tr.find_all('td'):
         row.append(td.text.strip())
     res.append(row)
     row = []

In [11]:
df = pd.DataFrame(data=res)
#df

In [12]:
#replace values not needed
df2=df.replace(to_replace= ["TRANSACTION TYPE"], value=' ', regex=True)
df22=df2.replace(to_replace= ["None"], value=' ', regex=True)
#df22

In [13]:
#drop rows of other lines
df3=df22.drop(df.index[[0,2,3,8,9,10,11,12,13,14,15,16,17,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70]])
#df3

In [14]:
#fill down values for line
for col in [11]:
    df3[col] = df3[col].ffill()

In [15]:
#Fix mis alignment of df bu using fillna caused by row/colspans in orginal html with if logic & finish filtering
df3[10] = df3[10].fillna(value=df3[9])
df3.loc[df3[1] == "20'",1] = df3[2] 
df3.loc[df3[2] == "",2] = df3[7] 
df3.loc[df3[3] == "",3] = df3[4] 
df4=df3.drop(columns=[0,4,5,6,7,8,9])
#print(df4)

In [16]:
#Change Headers for line and equip type
df4[10] = df4[10].fillna(value="Type")
df4[11] = df4[11].fillna(value="Line")
df4.head()

,1,2,3,10,11
1,5/29/2020,5/30/2020,5/31/2020,Type,Line
4,SINGLE,SINGLE,NO,20',MSC
5,SINGLE,DUAL,NO,40',MSC
6,SINGLE,DUAL,NO,40HC,MSC
7,SINGLE,DUAL,NO,45,MSC


In [17]:
# Set Headers
new_header = df4.iloc[0] 
df4 = df4[1:] 
df4.columns = new_header 
df4.head()

1,5/29/2020,5/30/2020,5/31/2020,Type,Line
4,SINGLE,SINGLE,NO,20',MSC
5,SINGLE,DUAL,NO,40',MSC
6,SINGLE,DUAL,NO,40HC,MSC
7,SINGLE,DUAL,NO,45,MSC
18,SINGLE,SINGLE,NO,20',MAE


In [19]:
#Reorder 
col_name="Type"
first_col = df4.pop(col_name)
df4.insert(0, col_name, first_col)
df4

1,Type,5/29/2020,5/30/2020,5/31/2020,Line
4,20',SINGLE,SINGLE,NO,MSC
5,40',SINGLE,DUAL,NO,MSC
6,40HC,SINGLE,DUAL,NO,MSC
7,45,SINGLE,DUAL,NO,MSC
18,20',SINGLE,SINGLE,NO,MAE
19,40',SINGLE,SINGLE,NO,MAE
20,40HC,SINGLE,SINGLE,NO,MAE
21,45,SINGLE,SINGLE,NO,MAE
42,20',DUAL,DUAL,NO,SML
43,40',DUAL,NO,NO,SML


In [20]:
#Reorder part duex....bc I did want to learn how to do it for both columns at once
col_name="Line"
first_col = df4.pop(col_name)
df4.insert(0, col_name, first_col)
df4

1,Line,Type,5/29/2020,5/30/2020,5/31/2020
4,MSC,20',SINGLE,SINGLE,NO
5,MSC,40',SINGLE,DUAL,NO
6,MSC,40HC,SINGLE,DUAL,NO
7,MSC,45,SINGLE,DUAL,NO
18,MAE,20',SINGLE,SINGLE,NO
19,MAE,40',SINGLE,SINGLE,NO
20,MAE,40HC,SINGLE,SINGLE,NO
21,MAE,45,SINGLE,SINGLE,NO
42,SML,20',DUAL,DUAL,NO
43,SML,40',DUAL,NO,NO


In [22]:
df4.to_csv(r"C:\Users\CodyS\Desktop\ETL-Project\TTI.csv", index = False)

In [28]:
dates = df4.iloc[:, [2,3,4]]

In [31]:
df5=df4.melt(id_vars=["Line","Type"], 
              var_name="Date", value_name="TTI Status")

In [32]:
df5.to_excel(r"C:\Users\CodyS\Desktop\ETL-Project\data.xlsx", index = False)
df5.to_csv(r"C:\Users\CodyS\Desktop\ETL-Project\TTI.csv", index = False)